# Introduction

These are my notes on an attempt to automate the integration of services/agents into the SingularityNET. The aim is to gather all the information necessary for service integration in a single file that can be filled via a user interface that makes the process intuitive and foolproof. 

> I understand that creating yet another info/config file would make things even more decentralized and confusing, but the idea here is for this to be the only file that the user should worry about filling. All the other info/config files will be generated from this one so that the service name/messages/ports are consistent.

That file will then be used to:

1. Register the service into SingularityNET
- Compile the protobuf message
- Create the 'service' package
- Automatically generate template code to run and test the service with commented examples.
- [Optional] Assemble command line calls for services given the arguments.
- [Optional] Automatic deployment using code from github and dockerfile

Even if automating all of the steps above isn't currently feasible (or too troublesome), some are definitely possible and would likely make service integration faster and clearer. Some thoughts on each step are detailed below.

# Config File Structure

-----IN PROGRESS-----

Probably a JSON file. Required fields:

1. For the protobuf message:

    - service name;
    - list of variable types
    - list of messages (using default and user defined variable types);
    - list of services (using messages as inputs);
    

2. For the service/service.json file (all the information required in 'snet service init'):

    - service name (already provided above);
    - path to the service's spec directory (default: "service_spec/");
    - an organization to register the service under: (default: "");
    - the path under which the Service registration will be created: (default: "");
    - a price in AGI to call the service (default: 0);
    - an endpoint to call the API for the service (default: "", e.g. http://54.203.198.53:7000 );
    - a list of tags for the service: (default: []);
    - a description for the service: (default: "");
    

3. For the './config/snetd_**service_name**_service.json':

    - "DAEMON_TYPE": "grpc", (default)
    - "DAEMON_LISTENING_PORT": "7000", (user provided)
    - "BLOCKCHAIN_ENABLED": true, (default)
    - "ETHEREUM_JSON_RPC_ENDPOINT": "https://kovan.infura.io" (user defined but this is the current default)
    - "AGENT_CONTRACT_ADDRESS": "YOUR_AGENT_ADDRESS", (get from service registration step)
    - "SERVICE_TYPE": "grpc", (default)
    - "PASSTHROUGH_ENABLED": true, (defaul)
    - "PASSTHROUGH_ENDPOINT": "http://localhost:7003", (user defined)
    - "LOG_LEVEL": 10, (user defined / default)
    - "PRIVATE_KEY": "YOUR_PRIVATE_KEY" (should probably get from an environment variable or something)
   
4. Extra information to allow for further automation steps

    - For each input:
        - required/optional;
        - optional default values;
        - optional for numeric variables: max / min values;
        - optional for strings: maxsize;
    - Extra [bash command](#bash-command) assembly parameters.

# Automation Steps

## 1) Register the service into SingularityNET

1. Ensure that the user has all the dependencies installed (snetd, snet-cli, grpc, protoc, etc.) and has properly created an identity (probably just check if 'identity_name' is defined);
- Create directories ('./config', './service');
- Generate the './service/service.json' file (containing information about the agent for the dApp). This step should be equivalent to 'snet service init';
- Create directory ('./service/service_spec') and generate the './service/service_spec/**service_name**.proto' file. (The user interface should also allow variable types other than the ones supported by protobuf, such as 'image'. Those will be encoded into base64 messages by the front-end and decoded in the back-end.)
- Generate the './config/snetd_**service_name**_service.json' (containing service endpoint, provider personal information, etc.). [This has to be well thought out since it needs sensitive information (user's private key). Maybe require that the user sets his private key as an environment variable and get it from there?]
- Use those files to register the service into SingularityNET. (Probably just 'snet service publish -y', though I'm not sure the '-y' flag currently exists for this command.)

> snet service publish -y

## 2) Compile the protobuf message

- Check that the inputs and outputs of the protobuf services have previously been defined as messages (or common types (int, bool, etc.) (?)). [This can be done in the front-end by only providing defined messages as options.]
- Compile the .proto message;

> python3.6 -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. service/service_spec/service_name.proto

## 3) Create the 'service' package

- Create './service/\__init__.py' to make Python treat the directory as containing packages. This is currently where the gRPC and daemon ports are defined:
>registry = {"**service_name**_service": {"grpc": 7022, "snetd": 7021}}
- Get the './service/serviceUtils.py' file (from GitHub ?) with utility code (base64 encoding, download, etc). This file currently imports 'registry' from "\__init__" to define the default grpc port in the parser [swtich to importing from the general config file?];
- Create a template for the './service/**service_name**_service.py' file.  This is where the actual gRPC servicer class is and where the main service code is called (currently through bash commands for lua, or via python's methods). Write the template code in a way that a new thread is generated for each request so that multiple requests are supported;
- [Optional] Treat inputs in './service/**service_name**_service.py' using functions from 'serviceUtils':
    - Check that required fields have been provided. **Remember that protobuf arguments are optional by default. If they're not provided, the default values for numeric types is 0, for booleans is false and for strings is the empty string "".**
    - Perform base64 conversion if necessary;
    - Put all the temporary files in the same folder with standardized names (temp_image_01, temp_image_02 ...). [Provide a function that creates that file in the temporary folder and returns its name.]
    - Make those files available to the service user for some time before being deleted. [Maybe let this time be a parameter.] This should probably be done through an Amazon Bucket

## 4) Automatically generate template code to run and test the service with commented examples.

- Create './run_**service_name**_service.py'

    1. Import the necessary modules (currently: pathlib, subprocess, time, os, sys, argparse, logging, threading and the registry (from service)), create a logger and an argument parser. 
    2. Get the list of service modules (from the config file);
    3. Remove previous snetd.db files;
    4. Start all the services. Loop through the list of service modules to:
        1. Get the service name, grpc port, snetdaemon configurations and db file;
        - Start the python module of the service at the specified gRPC port;
        - Create an instance of the daemon for the service;
    5. Serve forever (infinite loop).
    

- [OPTIONAL] Create './test_**service_name**_service.py'. This currently tests only the gRPC communication, not involving the blockchain.

    1. Import both of the (automatically generated) gRPC files, your service registry (in \__init__.py) and your model (in this case service.style_transfer);
    2. Get the gRPC endpoint from the registry file (localhost:endpoint);
    3. Create a gRPC insecure channel;
    4. Specify the gRPC method and parameters (in this case, the method is "transfer_style" and the parameters are the paths for both the content and the style images);
    5. Create a stub using the automatically generated class (in this case: style_transfer_rpc_pb2_grpc.StyleTransferStub);
    6. Create a request message (fill in the parameters of your message) using the input_info method of the automatically generated class;
    7. Make the call using stub.method(input_info);
    8. Treat the return if necessary.
    

## 5) [Optional]<a id='bash-command'> Bash command assembly parameters</a>  

(In './service/**service_name**_service.py'.)

- Get base string (bash command) to call each of the services defined (e.g.: base = 'th', image style transfer = 'test.lua', video style tranfer = 'testVid.lua')
- With or without value (ex: *-bool* or *-bool true*)
- One or two dashes (ex: *-bool* or *--bool* )
- Comas for multiple arguments (ex: single argument *-input_images image01* vs multiple arguments *-input_images image01,image02,...*)
- Assemble. Eg:
> th test.lua -inputs values

## 6) [Optional] Automatic deployment using code from github and dockerfile

TODO

# User Interface

TODO

___